[Ссылка на соревнование](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data)

In [3]:
import pandas as pd
import numpy as np

**File descriptions**

sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.

test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.

sample_submission.csv - a sample submission file in the correct format.

items.csv - supplemental information about the items/products.

item_categories.csv  - supplemental information about the items categories.

shops.csv- supplemental information about the shops.

**Data fields**

ID - an Id that represents a (Shop, Item) tuple within the test set

shop_id - unique identifier of a shop

item_id - unique identifier of a product

item_category_id - unique identifier of item category

item_cnt_day - number of products sold. You are predicting a monthly amount of this measure

item_price - current price of an item

date - date in format dd/mm/yyyy

date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33

item_name - name of item

shop_name - name of shop
item_category_name - name of item category

Начну с определия всех данных.

In [4]:
train_df = pd.read_csv('sales_train.csv')
test_df = pd.read_csv('test.csv')
items = pd.read_csv('items.csv')
items_cat = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')
samp = pd.read_csv('sample_submission.csv')

train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [5]:
test_df.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [6]:
samp.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


Обрежу данные для сортировки по месяцам, магазинам и вещам.

In [32]:
train_df['date'] = [date[3:5] for date in train_df['date']]
df = train_df.groupby(['date', 'shop_id', 'item_id']).agg(
    {'item_cnt_day': 'sum'}).reset_index()
df.head(15)

,date,shop_id,item_id,item_cnt_day
0,,0,30,31.0
1,,0,31,11.0
2,,0,32,16.0
3,,0,33,6.0
4,,0,35,15.0
5,,0,36,1.0
6,,0,40,1.0
7,,0,42,1.0
8,,0,43,1.0
9,,0,49,2.0


In [ ]:
Разобью данные на тренировочные и тестовые.

In [11]:
x = df[['shop_id', 'item_id']].values
y = df.item_cnt_day.values
test = test_df[['shop_id', 'item_id']].values

Обучение с помощью бустинга, так как даёт наилучшие результаты.

In [15]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

reg = xgb.XGBRegressor(booster='gblinear', random_state=10)
reg.fit(x, y)

[19:27:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gblinear', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=10,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [17]:
preds = reg.predict(test)

len(preds)

214200

Формирования итогового файла для Kaggle.

In [30]:
res = pd.DataFrame(preds)

res = res.rename(columns={0: 'item_cnt_month'})

res.head()

,item_cnt_month
0,2.277118
1,2.275920
2,2.276289
3,2.276293
4,2.276140


In [31]:
res.to_csv('submission.csv')

Мой итоговый **RMSE**: 2.57151.